In [1]:
# Data cleaning and engineering

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Removed: import plotly.express as px
from sklearn.preprocessing import OneHotEncoder, StandardScaler, FunctionTransformer
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import train_test_split, cross_val_score


class OutlierCapper(BaseEstimator, TransformerMixin):
    def __init__(self, lower_quantile=0.01, upper_quantile=0.99):
        self.lower_quantile = lower_quantile
        self.upper_quantile = upper_quantile
        self.lower_bounds = None
        self.upper_bounds = None

    def fit(self, X, y=None):
        self.lower_bounds = np.quantile(X, self.lower_quantile, axis=0)
        self.upper_bounds = np.quantile(X, self.upper_quantile, axis=0)
        return self

    def transform(self, X):
        return np.clip(X, self.lower_bounds, self.upper_bounds)

class YearConverter(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.min_year = None
        self.max_year = None

    def fit(self, X, y=None):
        X_flat = X.ravel() if X.ndim > 1 else X
        self.min_year = np.min(X_flat)
        self.max_year = min(np.max(X_flat), pd.Timestamp.now().year)
        return self

    def transform(self, X):
        X_numeric = pd.to_numeric(X.ravel() if X.ndim > 1 else X, errors='coerce')
        X_clipped = np.clip(X_numeric, self.min_year, self.max_year)
        return X_clipped.reshape(X.shape)

class FeatureEngineer(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        X = X.copy()
        # Create new features
        X['TotalSF'] = X['TotalBsmtSF'] + X['1stFlrSF'] + X['2ndFlrSF']
        
        current_year = pd.Timestamp.now().year
        X['HouseAge'] = current_year - X['YearBuilt']
        X['TimeSinceRemodel'] = current_year - X['YearRemodAdd']
        
        X['TotalBathrooms'] = X['FullBath'] + (0.5 * X['HalfBath']) + X['BsmtFullBath'] + (0.5 * X['BsmtHalfBath'])
        X['IsNewHouse'] = (X['YearBuilt'] == X['YrSold']).astype(int)
        X['HasPool'] = (X['PoolArea'] > 0).astype(int)
        X['TotalPorchSF'] = X['OpenPorchSF'] + X['EnclosedPorch'] + X['3SsnPorch'] + X['ScreenPorch']
        X['OverallHouseCondition'] = X['OverallQual'] * X['OverallCond']
        
        # Create interaction features
        X['TotalSF_OverallQual'] = X['TotalSF'] * X['OverallQual']
        X['GrLivArea_TotRmsAbvGrd'] = X['GrLivArea'] * X['TotRmsAbvGrd']
        X['HouseAge_OverallQual'] = X['HouseAge'] * X['OverallQual']
        X['GarageArea_GarageCars'] = X['GarageArea'] * X['GarageCars']
        X['YearBuilt_YearRemodAdd'] = X['YearBuilt'] * X['YearRemodAdd']
        X['TotalSF_HouseAge'] = X['TotalSF'] * X['HouseAge']
        X['1stFlrSF_2ndFlrSF'] = X['1stFlrSF'] * X['2ndFlrSF']
        X['TotalSF_OverallCond'] = X['TotalSF'] * X['OverallCond']
        
        # Interaction with categorical variable (requires encoding)
        X['GrLivArea_Neighborhood'] = X['GrLivArea'] * pd.factorize(X['Neighborhood'])[0]
        
        return X

def pandas_to_numpy(X):
    return X.to_numpy() if isinstance(X, pd.DataFrame) else X

def preprocess_and_engineer(X):
    # Apply FeatureEngineer first
    feature_engineer = FeatureEngineer()
    X_engineered = feature_engineer.fit_transform(X.copy())
    
    # Identify numeric, categorical, and year columns
    numeric_features = X_engineered.select_dtypes(include=['int64', 'float64']).columns.drop(['YearBuilt', 'YearRemodAdd', 'YrSold'])
    categorical_features = X_engineered.select_dtypes(include=['object']).columns
    year_features = ['YearBuilt', 'YearRemodAdd', 'YrSold']
    
    print("Number of features before preprocessing:")
    print(f"Numeric: {len(numeric_features)}")
    print(f"Categorical: {len(categorical_features)}")
    print(f"Year: {len(year_features)}")
    
    # Create preprocessing steps
    numeric_transformer = Pipeline(steps=[
        ('imputer', KNNImputer(n_neighbors=5)),
        ('outlier_capper', OutlierCapper()),
        ('scaler', StandardScaler()),
    ])

    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
        ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore', max_categories=10)),
    ])

    year_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('converter', YearConverter()),
    ])

    # Create and fit the preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features),
            ('year', year_transformer, year_features)
        ])
    
    X_preprocessed = preprocessor.fit_transform(X_engineered)
    
    # Generate feature names
    numeric_feature_names = list(numeric_features)
    categorical_feature_names = []
    onehot_encoder = preprocessor.named_transformers_['cat'].named_steps['onehot']
    
    print("\nCategorical feature encoding details:")
    for i, feature in enumerate(categorical_features):
        categories = onehot_encoder.categories_[i]
        n_categories = min(len(categories), 10)  # Account for max_categories=10
        n_encoded = n_categories - 1  # Subtract 1 due to drop='first'
        print(f"{feature}: {n_categories} categories, {n_encoded} encoded features")
        categorical_feature_names.extend([f"{feature}_{cat}" for cat in categories[1:n_categories]])
    
    year_feature_names = list(year_features)
    
    feature_names = (numeric_feature_names + 
                     categorical_feature_names + 
                     year_feature_names)
    
    print("\nNumber of features after preprocessing:")
    print(f"Numeric: {len(numeric_feature_names)}")
    print(f"Categorical (one-hot encoded): {len(categorical_feature_names)}")
    print(f"Year: {len(year_feature_names)}")
    
    print(f"\nTotal number of features: {len(feature_names)}")
    print(f"Number of columns in preprocessed data: {X_preprocessed.shape[1]}")
    
    # Ensure the number of feature names matches the number of columns in X_preprocessed
    if len(feature_names) != X_preprocessed.shape[1]:
        print(f"\nWarning: Number of feature names ({len(feature_names)}) "
              f"does not match number of columns in preprocessed data ({X_preprocessed.shape[1]})")
        print("Adjusting feature names...")
        if len(feature_names) > X_preprocessed.shape[1]:
            feature_names = feature_names[:X_preprocessed.shape[1]]
        else:
            feature_names += [f'Unknown_{i}' for i in range(X_preprocessed.shape[1] - len(feature_names))]
    
    # Store feature names as an attribute of the DataFrame
    df = pd.DataFrame(X_preprocessed, columns=feature_names, index=X.index)
    df.attrs['feature_names'] = feature_names
    
    return df

# Load the data
df = pd.read_csv('/Users/ttanaka/Desktop/Website/house-prices-advanced-regression-techniques/train.csv')

# Separate features and target
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

# Full pipeline
full_pipeline = Pipeline([
    ('preprocess_and_engineer', FunctionTransformer(preprocess_and_engineer, validate=False)),
    ('to_numpy', FunctionTransformer(pandas_to_numpy))
])

# Apply the pipeline
X_processed = full_pipeline.fit_transform(X)

# Validation
print("\nFinal validation:")
print("Shape after preprocessing:", X_processed.shape)
print("Missing values after preprocessing:", np.isnan(X_processed).sum())

# Split the processed data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

# Access feature names from the intermediate DataFrame
intermediate_df = full_pipeline.named_steps['preprocess_and_engineer'].transform(X)
feature_names = intermediate_df.attrs.get('feature_names', [])
print("Number of features:", len(feature_names))
print("First 10 feature names:", feature_names[:10])
print("Last 10 feature names:", feature_names[-10:])

Number of features before preprocessing:
Numeric: 51
Categorical: 43
Year: 3

Categorical feature encoding details:
MSZoning: 5 categories, 4 encoded features
Street: 2 categories, 1 encoded features
Alley: 3 categories, 2 encoded features
LotShape: 4 categories, 3 encoded features
LandContour: 4 categories, 3 encoded features
Utilities: 2 categories, 1 encoded features
LotConfig: 5 categories, 4 encoded features
LandSlope: 3 categories, 2 encoded features
Neighborhood: 10 categories, 9 encoded features
Condition1: 9 categories, 8 encoded features
Condition2: 8 categories, 7 encoded features
BldgType: 5 categories, 4 encoded features
HouseStyle: 8 categories, 7 encoded features
RoofStyle: 6 categories, 5 encoded features
RoofMatl: 8 categories, 7 encoded features
Exterior1st: 10 categories, 9 encoded features
Exterior2nd: 10 categories, 9 encoded features
MasVnrType: 4 categories, 3 encoded features
ExterQual: 4 categories, 3 encoded features
ExterCond: 5 categories, 4 encoded features

In [19]:
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, StackingRegressor, VotingRegressor
from sklearn.linear_model import ElasticNet
from sklearn.svm import SVR
import numpy as np

# Define models
rf = RandomForestRegressor(n_estimators=100, random_state=42)
gb = GradientBoostingRegressor(n_estimators=100, random_state=42)
svm = SVR(kernel='linear', C=834.2988013047346, epsilon=0.7722447692966574, gamma=0.0006235377135673155)
en = ElasticNet(random_state=42)

estimators = [
    ('rf', rf),
    ('gb', gb),
    ('svm', svm),
    ('en', en)
]

stacking_regressor = StackingRegressor(
    estimators=estimators,
    final_estimator=ElasticNet(random_state=42)
)

voting_regressor = VotingRegressor(estimators=estimators)

models = [rf, gb, svm, en, stacking_regressor, voting_regressor]
model_names = ['Random Forest', 'Gradient Boosting', 'SVM (tuned)', 'Elastic Net', 'Stacking', 'Voting']


# Perform cross-validation and calculate metrics
for name, model in zip(model_names, models):
    mae_scores = -cross_val_score(model, X_processed, y, scoring='neg_mean_absolute_error', cv=5)
    mse_scores = -cross_val_score(model, X_processed, y, scoring='neg_mean_squared_error', cv=5)
    r2_scores = cross_val_score(model, X_processed, y, scoring='r2', cv=5)
    
    # Calculate MAPE and MedAE using custom scorers
    def mape_scorer(estimator, X, y):
        y_pred = estimator.predict(X)
        return np.mean(np.abs((y - y_pred) / y)) * 100

    def medae_scorer(estimator, X, y):
        y_pred = estimator.predict(X)
        return median_absolute_error(y, y_pred)

    mape_scores = cross_val_score(model, X_processed, y, scoring=mape_scorer, cv=5)
    medae_scores = cross_val_score(model, X_processed, y, scoring=medae_scorer, cv=5)
    
    print(f"{name}:")
    print(f"MAE: {mae_scores.mean():.1f} (±{mae_scores.std() * 2:.1f})")
    print(f"MSE: {mse_scores.mean():.1f} (±{mse_scores.std() * 2:.1f})")
    print(f"RMSE: {np.sqrt(mse_scores.mean()):.1f} (±{np.sqrt(mse_scores.std() * 2):.1f})")
    print(f"MAPE: {mape_scores.mean():.2f}% (±{mape_scores.std() * 2:.2f}%)")
    print(f"MedAE: {medae_scores.mean():.1f} (±{medae_scores.std() * 2:.1f})")
    print(f"R²: {r2_scores.mean():.3f} (±{r2_scores.std() * 2:.3f})")
    print()

Random Forest:
MAE: 17577.4 (±2388.1)
MSE: 936744142.0 (±550308478.5)
RMSE: 30606.3 (±23458.7)
MAPE: 10.14% (±1.06%)
MedAE: 10972.8 (±1882.2)
R²: 0.851 (±0.072)

Gradient Boosting:
MAE: 16361.4 (±1846.1)
MSE: 841185633.4 (±453050944.9)
RMSE: 29003.2 (±21285.0)
MAPE: 9.43% (±1.08%)
MedAE: 10081.1 (±1158.8)
R²: 0.868 (±0.052)

SVM (tuned):
MAE: 16190.0 (±2751.3)
MSE: 854546628.4 (±613610477.2)
RMSE: 29232.6 (±24771.2)
MAPE: 9.43% (±1.09%)
MedAE: 10484.0 (±1913.6)
R²: 0.867 (±0.071)

Elastic Net:
MAE: 17998.6 (±2671.6)
MSE: 971907418.5 (±643087728.2)
RMSE: 31175.4 (±25359.2)
MAPE: 10.24% (±1.07%)
MedAE: 12387.4 (±1903.4)
R²: 0.848 (±0.073)



/Users/ttanaka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.021e+10, tolerance: 7.592e+08
  model = cd_fast.enet_coordinate_descent(
/Users/ttanaka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.523e+10, tolerance: 7.288e+08
  model = cd_fast.enet_coordinate_descent(
/Users/ttanaka/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regula

Stacking:
MAE: 15816.4 (±2518.0)
MSE: 796930261.8 (±597972816.7)
RMSE: 28230.0 (±24453.5)
MAPE: 9.31% (±1.10%)
MedAE: 10275.4 (±1337.1)
R²: 0.876 (±0.073)

Voting:
MAE: 15576.9 (±2252.3)
MSE: 795293683.3 (±525911764.7)
RMSE: 28201.0 (±22932.8)
MAPE: 8.91% (±0.97%)
MedAE: 10029.2 (±1607.8)
R²: 0.876 (±0.061)

